In [12]:
import autogen
import os

In [ ]:
api_key = os.environ.get("GROQ_API_KEY")

config_list = [
  {
    "model": "llama-3.2-3b-preview",
    "base_url": "https://api.groq.com/openai/v1/chat/completions",
    "api_type": "groq",
    "api_key": api_key,
    "api_rate_limit": 2,
  }
]

llm_config = config_list[0]

In [14]:
task = "Write a blogpost about the stock price performance of "\
"Nvidia in the past month. Today's date is 2024-04-23."

In [15]:
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Give the task, and send "
    "instructions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

In [16]:
planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine "
    "what information is needed to complete the task. "
    "Please note that the information will all be retrieved using"
    " Python code. Please only suggest information that can be "
    "retrieved using Python code. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps. If a step fails, try to "
    "workaround",
    description="Planner. Given a task, determine what "
    "information is needed to complete the task. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps",
    llm_config=llm_config,
)

In [17]:
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="An engineer that writes code based on the plan "
    "provided by the planner.",
)

In [18]:
executor = autogen.ConversableAgent(
    name="Executor",
    system_message="Execute the code written by the "
    "engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
)

In [19]:
writer = autogen.ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Writer."
    "Please write blogs in markdown format (with relevant titles)"
    " and put the content in pseudo ```md``` code block. "
    "You take feedback from the admin and refine your blog.",
    description="Writer."
    "Write blogs based on the code execution results and take "
    "feedback from the admin to refine the blog."
)

In [ ]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
)

In [21]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)

In [22]:
groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

Admin (to chat_manager):

Write a blogpost about the stock price performance of Nvidia in the past month. Today's date is 2024-04-23.

--------------------------------------------------------------------------------


IndexError: list index out of range

In [2]:
import os
from autogen import AssistantAgent, UserProxyAgent

api_key = os.environ.get("GROQ_API_KEY")

config_list = [
  {
    "model": "llama-3.2-3b-preview",
    "base_url": "https://api.groq.com/openai/v1/chat/completions",
    "api_type": "groq",
    "api_key": api_key,
  }
]

assistant = AssistantAgent("assistant", llm_config={"config_list": config_list})

user_proxy = UserProxyAgent("user_proxy", code_execution_config={"work_dir": "coding", "use_docker": False})
chat_result = user_proxy.initiate_chat(assistant, message="Plot a chart of NVDA and TESLA stock price change YTD.")

user_proxy (to assistant):

Plot a chart of NVDA and TESLA stock price change YTD.

--------------------------------------------------------------------------------
assistant (to user_proxy):

# filename: stock_price_change.py

```python
# Import necessary libraries
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

# Define the stock symbols
stock_symbols = ['NVDA', 'TSLA']

# Get the latest data for the given stocks
data = {stock: yf.download(stock, period='2023-01-01', interval='1d')['Adj Close'] for stock in stock_symbols}

# Create a DataFrame from the data
df = pd.DataFrame(list(data.values()), columns=stock_symbols)

# Calculate the YTD percentage change
df['YTD'] = (df.iloc[:, 0] - df.iloc[0, 0]) / df.iloc[0, 0] * 100

# Plot the chart
plt.figure(figsize=(10,6))
plt.plot(df.iloc[:, 0], label='NVDA')
plt.plot(df.iloc[:, 1], label='TESLA')
plt.title('Stock Price Change (YTD)')
plt.xlabel('Date')
plt.ylabel('Price ( Adj Close)')
plt.legend()
plt.show()
```


c:\Users\ahmad\Desktop\LlamaHackers\venv\Lib\site-packages\autogen\oai\groq.py:280: UserWarning: Cost calculation not available for model llama-3.2-3b-preview
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


In [5]:
chat_result.summary

"# filename: stock_price_change.py\n\n```python\n# Import necessary libraries\nimport requests\nimport json\nimport pandas as pd\nimport matplotlib.pyplot as plt\n\n# Define the stock symbols\nstock_symbols = ['NVDA', 'TSLA']\n\n# Define the API URL\nurl = 'https://FinancialModelingPrep.com/api/v3/input'\n\n# Set the API key (replace with your actual API key)\napi_key = 'YOUR_API_KEY_HERE'\n\n# Get the latest data for the given stocks using the API\nheaders = {\n    'apikey': api_key,\n    'function': 'VTDatenShowAll'\n}\ndata = {\n    'Ticker': ','.join(stock_symbols),\n    'StartPeriod': '2023-01-01'\n}\nresponse = requests.post(url, headers=headers, json=data)\n\n# Check if the API request was successful\nif response.status_code == 200:\n    # Parse the JSON response\n    data = response.json()\n\n    # Extract the daily prices for the given stocks\n    daily_prices = data['Result']['Data']['NVDA']['DailyPrices']\n    tesla_daily_prices = data['Result']['Data']['TSLA']['DailyPrices'